In [22]:
from canvasapi import Canvas
import os
from datetime import datetime
from  pathlib import Path
from bs4 import BeautifulSoup

In [23]:
def read_secret_token(file_path):
    with open(file_path, 'r') as file:
        token = file.read().strip()
    return token

def initialize_canvas(api_url, api_key):
    """ initialize the Canvas object. """
    return Canvas(api_url, api_key)

def update_assignment(assignment, description):
    assignment.edit(assignment={'description': description})



def create_assignment(course, name, description):
    new_assignment = course.create_assignment({
        'name': name,
        'submission_types': ['online_upload', "online_text_entry"],
        'allowed_extensions': ['docx', 'doc', 'pdf', "", "txt"],
        'notify_of_update': True,
        'description': description,
        'published': True
    })
    return new_assignment

def get_body(file):
    with open(file, 'r') as f:
        body = f.read()
        # Parse the HTML content
        soup = BeautifulSoup(body, 'html.parser')

        # Find all the links and print their href attributes
        for link in soup.find_all('a'):
            href = link.get('href')
            if href and href.startswith('../'):
                link['href'] = 'https://normrubin.github.io/' + href[3:]

            # Convert the modified soup back to a string
        updated_body = str(soup)
    return updated_body



In [24]:
def clear_assignments(course):
    for a in course.get_assignments():
        a.delete()


In [25]:
def copy_homework_files_to_canvas(course, hw_files_path):
    hw_files = Path(hw_files_path)
    assignments = {a.name: a for a in course.get_assignments()}

    for count, file in enumerate(hw_files.glob("*.html")):
        name = f"Homework {file.stem}"
        file_mod_time = datetime.fromtimestamp(os.path.getmtime(file))
        if name in assignments:
            assignment = assignments[name]
            assignment_mod_time = datetime.strptime(assignment.created_at, '%Y-%m-%dT%H:%M:%SZ')
            if file_mod_time > assignment_mod_time:
                body = get_body(file)
                with open(file, 'r') as f:
                    body = f.read()
                    update_assignment(assignment, body)
                    print(f"Updated Assignment: {name}")
            else:
                print(f"Assignment {name} is up to date.")
        else:
            # new assignment 
            body = get_body(file)
            create_assignment(course, name, body)
            print(f"Created Assignment: {name}")

In [26]:
# Main execution
API_KEY = read_secret_token('/home/norm/canvas-token.txt')

## test location
API_URL = "https://northeastern.instructure.com/"
COURSE_ID = 198229  

canvas = initialize_canvas(API_URL, API_KEY)
course = canvas.get_course(COURSE_ID)

#   clear_assignments(course)
# list_course_pages(course)
copy_homework_files_to_canvas(course, "../_site/homework")

# discussions 
# we want one for each active paper 
# one for every class in sym 

Updated Assignment: Homework 5_hw
Updated Assignment: Homework project
Updated Assignment: Homework 4_hw
Updated Assignment: Homework hw0
Updated Assignment: Homework 6_extra_credit
Updated Assignment: Homework 3_hw
Updated Assignment: Homework dynamic
Updated Assignment: Homework 1.hw
Updated Assignment: Homework 2_hw
